#### This is to etract decscription content from play 2 play data

In [4]:
import pandas as pd
import numpy as np
import requests
import data_getter as dg
import time
import os
# pd.options.display.max_rows = 999
# pd.reset_option("display.max_rows")
pd.get_option("display.max_columns")
pd.set_option("display.max_columns", 999)

In [5]:
# Get 2017-18 season games list
boxscore_list_df = dg.get_boxscore_list_to_df()

In [6]:
game_id_list = list(boxscore_list_df.GAME_ID.unique())
len(game_id_list)

752

In [7]:
# Download pp data as dataframe.
pp_df_dict = {}
for i in game_id_list:
    filename = "data/pp/"+"pp_"+str(i)+".csv"
    if os.path.isfile(filename):
        pp_df = pd.read_csv(filename)
    else:   
        pp_df = dg.get_one_game_play_to_play_to_df(game_id=i) 
        pp_df.to_csv("data/pp/"+"pp_"+str(i)+".csv", header=True, index=False)
    pp_df_dict[i] = pp_df
    time.sleep(0.15)

#### Save data to pickle.

In [5]:
import pickle
import datetime

In [6]:
date_str = datetime.datetime.today().strftime('%Y%m%d')
print(date_str)
pp_dump_file_name = "parm_and_plays_list_"+str(date_str)+".p"
print(os.path.join('data', pp_dump_file_name))
pickle.dump((game_id_list, pp_df_dict), open(os.path.join('data', pp_dump_file_name), 'wb'))

20180128
data/parm_and_plays_list_20180128.p


#### Checkpoint to read out parm

In [8]:
import pandas as pd
import numpy as np
import requests
import data_getter as dg
import time
import os
import pickle
game_id_list, pp_df_dict = pickle.load(open('data/parm_and_plays_list_20180128.p', 'rb'))

In [9]:
main_column_list = ['PLAYER1_NAME', 'PLAYER2_NAME', 'PLAYER3_NAME', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION',
       'VISITORDESCRIPTION']

In [10]:
description_text = ""
for game_id, game_df in pp_df_dict.items():
# for game_id, game_df in test_df_dict.items():
    game_pp_text = dg.get_one_pp_to_text(game_df)
    # game_pp_text = get_one_pp_to_text(game_df)
    start_text = "GAME_START\n"
    end_text = "GAME_OVER"
    full_game_pp_text = start_text + game_pp_text + end_text
    # full_game_pp_text = "\n".join([start_text, game_pp_text, end_text])
    description_text = "\n\n".join([description_text, full_game_pp_text])

In [11]:
games_count = len(pp_df_dict)
pp_description_text_file_name = "games_count_and_description_text.p"
pickle.dump((games_count, description_text), open(os.path.join('data', pp_description_text_file_name), 'wb'))

#### Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [12]:
text = description_text
view_play_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
games = text.split('\n\n')
print('Number of games: {}'.format(len(games)))
play_count_game = [game.count('\n') for game in games]
print('Average number of games in each game: {}'.format(np.average(play_count_game)))

plays = [play for game in games for play in game.split('\n')]
print('Number of lines: {}'.format(len(plays)))
word_count_play = [len(play.split()) for play in plays]
print('Average number of words in each line: {}'.format(np.average(word_count_play)))

print()
print('The plays {} to {}:'.format(*view_play_range))
print('\n'.join(text.split('\n')[view_play_range[0]:view_play_range[1]]))


Dataset Stats
Roughly the number of unique words: 79794
Number of games: 731
Average number of games in each game: 457.47606019151846
Number of lines: 335146
Average number of words in each line: 6.964284222398597

The plays 0 to 10:


GAME_START
,,,,,
Zaza Pachulia,Al Horford,Kyrie Irving,Jump Ball Pachulia vs. Horford: Tip to Irving,,
Jaylen Brown,Al Horford,,,,Brown 1' Driving Dunk (2 PTS) (Horford 1 AST)
Klay Thompson,,,MISS Thompson 19' Jump Shot,,
Jaylen Brown,,,,,Brown REBOUND (Off:0 Def:1)
Al Horford,Kyrie Irving,,,,Horford 26' 3PT Jump Shot (3 PTS) (Irving 1 AST)
Stephen Curry,,,Curry 11' Floating Jump Shot (2 PTS),,


#### Tokenizer the text

In [14]:
from keras.preprocessing.text import Tokenizer

In [15]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token = dict()
    # token['.'] = "||Period||"
    token[','] = "DESC_SEP"
    # token['"'] = "||Quotation_Mark||"
    # token[';'] = "||Semicolon||"
    # token['!'] = "||Exclamation_Mark||"
    # token['?'] = "||Question_Mark||"
    # token['('] = "||Left_Parentheses||"
    # token[')'] = "||Right_Parentheses||"
    # token['--'] = "||Dash||"
    token['\n'] = "LINE_SEP"
    # token['\n\n'] = "GAME_SEP"

    return token

token_dict = token_lookup()
token_text = text
for key, token in token_dict.items():
    token_text = token_text.replace(key, ' {} '.format(token))


text_list = [token_text]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
tokenizer.fit_on_texts(text_list)
sequences = tokenizer.texts_to_sequences(text_list)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 1410 unique tokens.


In [16]:
char_indices = tokenizer.word_index
indices_char = {v: k for k, v in char_indices.items()}

In [17]:
# prepare the dataset of input to output pairs encoded as integers
vocab_size = len(word_index) + 1
char_size = len(sequences[0])
int_text = sequences[0]
seq_length = 15
dataX = []
dataY = []

for i in range(0, char_size - seq_length, 1):
    dataX.append(int_text[i:i + seq_length])
    dataY.append(int_text[i+seq_length])
n_patterns = len(dataX)
print("Total Patterns: {}\n dataX shape is {}\n dataY shape is {}".format(n_patterns, np.shape(dataX), np.shape(dataY)))


Total Patterns: 5033011
 dataX shape is (5033011, 15)
 dataY shape is (5033011,)


##### Without embedding

In [16]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras_tqdm import TQDMNotebookCallback

# reshape X to be [samples, time steps, features]
no_embed_X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
no_embed_X = no_embed_X / float(vocab_size)
# one hot encode the output variable
no_embed_y = np_utils.to_categorical(dataY)

In [17]:
# define the LSTM model
model = Sequential()
# model.add(Embedding(max_features, 128))
model.add(LSTM(128, input_shape=(no_embed_X.shape[1], no_embed_X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, TQDMNotebookCallback()]
# fit the model

model.fit(no_embed_X, no_embed_y, epochs=1, batch_size=128, verbose=0, callbacks=callbacks_list)

Epoch 1/1
  31104/3364586 [..............................] - ETA: 1:46:46 - loss: 5.7223 

#### With embedding

In [18]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.callbacks import LambdaCallback

embed_x = np.array(dataX)
embed_y = np.array(dataY)
s_embed_y = np_utils.to_categorical(dataY)

In [25]:
print("input_x shape is {} \ninput_y shape is {}".format(np.shape(embed_x), np.shape(s_embed_y)))
print("input_x[0] is {} \ninput_y[0] is {}".format(embed_x[2], indices_char[np.argmax(s_embed_y[2])]))

input_x shape is (5033011, 15) 
input_y shape is (5033011, 1411)
input_x[0] is [659   2   1   1   1   1   1   2 638 447   1 185 169   1 311] 
input_y[0] is irving


In [12]:


# define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 128))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [32]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [13]:


def on_epoch_end(epoch, logs):
    pass

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

from keras_tqdm import TQDMNotebookCallback
model.fit(embed_x, s_embed_y,
          batch_size=128,
          epochs=10,
         verbose=0, callbacks=[TQDMNotebookCallback()])

In [14]:
model.save('pp_model_20180129_10_epoch.h5')

#### Checkpoint

In [3]:
from keras.models import load_model

model = load_model('pp_model_20180129_10_epoch.h5')

/Users/ccuulinay/.pyenv/versions/anaconda3-4.2.0/envs/dlnd/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/ccuulinay/.pyenv/versions/anaconda3-4.2.0/envs/dlnd/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [43]:
def sample_word(preds, indices_char,temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.squeeze(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return indices_char[np.argmax(probas)]


gen_length = 200
prime_word = 'game_start'

gen_sentences = [prime_word]

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print('----- diversity:', diversity)
    
    generated = gen_sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = np.array([[char_indices[word] for word in gen_sentences[-seq_length:]]])
        pred=model.predict(dyn_input)
        word=sample_word(pred, indices_char)
        generated.append(word)
    pp_script = ' '.join(generated)
    for key, token in token_dict.items():
        pp_script = pp_script.replace(' ' + token.lower(), key)
    print(pp_script)


----- diversity: 0.2


/Users/ccuulinay/.pyenv/versions/anaconda3-4.2.0/envs/dlnd/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log


game_start jerebko game_over mclemore ellenson williams jump shot 5 pts young 2 ast,,
 cody zeller, marco belinelli,, lamb steal 1 stl,, zeller bad pass turnover p2 t14
 jeremy lamb,,, lamb 18' jump shot 12 pts,,
 cory joseph,,,,, miss joseph 11' turnaround jump shot
 troy daniels,,, daniels rebound off 0 def 2,,
 danuel house,,, house 7' hook shot 4 pts,,
 jimmy butler,,,,, butler 14' jump shot 11 pts
 kyrie irving,,, miss irving 27' 3pt jump shot,,
 kyrie irving,,, irving rebound off 1 def 5,,
 kyrie irving, jaylen brown,,,, sub irving for brown
 iman shumpert,,, miss shumpert free throw 2 of 2,,
 dillon brooks,,,,, brooks rebound off 0 def 3
 kentavious caldwell pope, jordan clarkson,,,, caldwell pope 26' 3pt jump shot 18 pts clarkson 1 ast
 alex caruso
----- diversity: 0.5
game_start jerebko game_over mclemore ellenson williams jump shot 5 pts young 2 ast,,
 cody zeller, marco belinelli,, lamb steal 1 stl,, zeller bad pass turnover p2 t14
 jeremy lamb,,, lamb 18' jump shot 12 pts,,